In [139]:
# Import supporting packages
import openpyxl
from bs4 import BeautifulSoup
import re
import requests
import os
import pandas as pd
import numpy as np
from pandas import ExcelWriter
from pandas import ExcelFile
!pip install PyGithub
from github import Github
import urllib
import os
import matplotlib as mpl
import matplotlib.pyplot as plt
import math
from datetime import datetime
import datetime as dt
import time
import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

import json
from ast import literal_eval

print('Libraries imported.')

You are using pip version 18.1, however version 19.0 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries imported.


In [140]:
import fuzzymatcher
import fuzzywuzzy
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [141]:
df = pd.read_pickle('features_data_all_vars.pkl')

In [142]:
df = df[df.columns.drop(['best_match_score', '__id_left', '__id_right','city_right','state_right','GeoFips','GeoName','2017'])]


In [143]:
df = df.rename(columns={'city_left': 'city', 'state_left': 'state'})

In [144]:
df.columns

Index(['city', 'state', 'pct_time_in_traffic', 'bus_per_sq_mi',
       'percip_inches', 'density2019', 'state_tax', 'city_tax', 'total_tax',
       'year_F', 'walk_score', 'transit_score', 'bike_score', 'population',
       'gdp_per_cap_17'],
      dtype='object')

In [145]:
geo_data = pd.read_csv('1000-largest-us-cities-by-population-with-geographic-coordinates.csv', sep = ';')

In [146]:
geo_data.columns

Index(['City', 'Rank', 'State', 'Growth From 2000 to 2013', 'Population',
       'Coordinates'],
      dtype='object')

In [147]:
geo_data['State'] = [x.lower() for x in geo_data['State']]
geo_data['city'] = [x.lower() for x in geo_data['City']]


In [148]:
geo_data['state'] = geo_data['State'].map(states_abbrev_2)


In [149]:
geo_data[['latitude','longitude']] = geo_data['Coordinates'].str.split(',', n=1, expand=True)
geo_data.head()


,City,Rank,State,Growth From 2000 to 2013,Population,Coordinates,city,state,latitude,longitude
0,South San Francisco,522,california,9.1,66174,"37.654656, -122.4077498",south san francisco,CA,37.654656,-122.4077498
1,Aliso Viejo,741,california,25.4,50175,"33.5676842, -117.7256083",aliso viejo,CA,33.5676842,-117.7256083
2,Rapid City,479,south dakota,17.9,70812,"44.0805434, -103.2310149",rapid city,SD,44.0805434,-103.2310149
3,Coon Rapids,563,minnesota,0.6,62103,"45.1732394, -93.3030063",coon rapids,MN,45.1732394,-93.3030063
4,Malden,585,massachusetts,7.4,60509,"42.4250964, -71.066163",malden,MA,42.4250964,-71.066163


In [150]:
geo_data.latitude = [float(x) for x in geo_data.latitude]
geo_data.longitude = [float(x) for x in geo_data.longitude]
geo_data.Rank = [float(x) for x in geo_data.Rank]
geo_data['pop_quartile'] = pd.qcut(geo_data['Population'], 4, labels=False)

In [151]:
df_all = fuzzymatcher.fuzzy_left_join(df, geo_data, left_on=['city','state'], right_on=['city','state'])

In [152]:
df_all = df_all[df_all.best_match_score >= 0.1]
len(df_all)

252

In [153]:
df_all['pct_time_in_traffic_quart'] = pd.qcut(df_all['Population'], 4, labels=False)
df_all['percip_inches_quart'] = pd.qcut(df_all['percip_inches'], 4, labels=False)
df_all['bike_quart'] = pd.qcut(df_all['bike_score'], 4, labels=False)

In [154]:
df_all = df_all.rename(columns={'city_left': 'city', 'state_left': 'state'})

In [173]:
latitude = 37.0902
longitude = -95.7129
traffic_map = folium.Map(location=[latitude, longitude], zoom_start=5)

colordict = {0.0: 'lightblue', 1.0: 'lightgreen', 2.0: 'orange', 3.0: 'red'}
#rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, traffic, bike, city in zip(df_all['latitude'], df_all['longitude'], 
                                     df_all['pct_time_in_traffic_quart'], df_all['bike_score'], 
                                     df_all['city']):
    label = folium.Popup(str(city), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=.2*bike,
        popup=label,
        color='b',
        key_on = traffic,
        threshold_scale=[0,1,2,3],
        fill_color=colordict[traffic],
        #fill_color = 'blue',
        fill=True,
        fill_opacity=0.7
        ).add_to(traffic_map)

    
traffic_map


In [156]:
df_all.columns

Index(['best_match_score', '__id_left', '__id_right', 'city', 'state',
       'pct_time_in_traffic', 'bus_per_sq_mi', 'percip_inches', 'density2019',
       'state_tax', 'city_tax', 'total_tax', 'year_F', 'walk_score',
       'transit_score', 'bike_score', 'population', 'gdp_per_cap_17', 'City',
       'Rank', 'State', 'Growth From 2000 to 2013', 'Population',
       'Coordinates', 'city_right', 'state_right', 'latitude', 'longitude',
       'pop_quartile', 'pct_time_in_traffic_quart', 'percip_inches_quart',
       'bike_quart'],
      dtype='object')

In [170]:
df_clean = df_all[['pct_time_in_traffic', 'bus_per_sq_mi', 'percip_inches', 'density2019',
       'state_tax', 'city_tax', 'total_tax', 'year_F', 'walk_score',
       'transit_score', 'bike_score', 'population', 'gdp_per_cap_17']]

In [171]:
df_clean = df_clean.rename(columns={'pct_time_in_traffic': 'Traffic index', 'bus_per_sq_mi': 'Business density',
                                    'percip_inches':'Average rainfall', 'density2019': 'Population density',
                                    'state_tax': 'State taxes', 'city_tax':'City taxes', 
                                    'total_tax': 'Local taxes', 
                                    'year_F': 'Average temperature', 
                                    'walk_score': 'Walk score',
                                    'transit_score': 'Transit score', 
                                    'bike_score': 'Bike score', 
                                    'population': 'Population', 
                                    'gdp_per_cap_17': 'GDP per capita'
                                   })

***

In [ ]:
fig, ax = plt.subplots(figsize=(10,10)) 
sns.heatmap(df.corr(), cmap="seismic", annot=True, vmin=-1, vmax=.8, ax=ax);



In [157]:
cols = ['pct_time_in_traffic', 'bus_per_sq_mi', 'percip_inches', 'density2019',
       'state_tax', 'city_tax', 'total_tax', 'year_F', 'walk_score',
       'transit_score', 'bike_score', 'population', 'gdp_per_cap_17']
for n in cols:
    df_all[n] = df_all[n].apply(pd.to_numeric, errors = 'coerce')


df_all[cols] = df_all[cols].apply(lambda x: x.fillna(x.mean()),axis=0)

In [158]:
df_all.percip_inches_quart

0       0.0
1       3.0
37      0.0
39      3.0
40      3.0
45      NaN
46      2.0
47      1.0
48      1.0
49      1.0
50      3.0
51      2.0
53      0.0
54      1.0
55      3.0
56      1.0
57      1.0
58      2.0
59      0.0
60      0.0
61      1.0
62      3.0
63      0.0
64      3.0
65      2.0
66      2.0
67      2.0
68      1.0
69      0.0
71      0.0
       ... 
1306    1.0
1307    1.0
1308    3.0
1309    1.0
1310    3.0
1311    3.0
1312    0.0
1313    3.0
1323    1.0
1324    NaN
1325    1.0
1326    NaN
1327    2.0
1328    1.0
1329    NaN
1341    2.0
1342    2.0
1343    NaN
1394    1.0
1395    NaN
1396    1.0
1397    3.0
1398    3.0
1400    3.0
1419    2.0
1420    2.0
1421    1.0
1422    0.0
1436    NaN
1437    2.0
Name: percip_inches_quart, Length: 252, dtype: float64

In [107]:
df_all[df_all.percip_inches_quart.isna() == True]

,best_match_score,__id_left,__id_right,city,state,pct_time_in_traffic,bus_per_sq_mi,percip_inches,density2019,state_tax,...,Growth From 2000 to 2013,Population,Coordinates,city_right,state_right,latitude,longitude,pop_quartile,pct_time_in_traffic_quart,percip_inches_quart


In [166]:
df_test = df_all
cities = ['columbus','atlanta']
for c in cities:
    df_test.loc[df_test['city'].str.contains(c), 'New_Col_1'] = c


In [167]:
df_test[df_test['New_Col_1'].isna() == False]

,best_match_score,__id_left,__id_right,city,state,pct_time_in_traffic,bus_per_sq_mi,percip_inches,density2019,state_tax,...,Coordinates,city_right,state_right,latitude,longitude,pop_quartile,pct_time_in_traffic_quart,percip_inches_quart,bike_quart,New_Col_1
39,0.279038,3_left,840_right,atlanta,GA,0.10,30.161538,49.70,1450.0,0.0400,...,"33.7489954, -84.3879824",atlanta,GA,33.748995,-84.387982,3,3,3.0,1.0,atlanta
74,0.262688,31_left,798_right,columbus,OH,0.06,30.161538,39.30,1572.0,0.0575,...,"39.9611755, -82.9987942",columbus,OH,39.961176,-82.998794,3,3,1.0,2.0,columbus
1220,0.257264,240_left,237_right,columbus,GA,0.04,30.161538,46.76,341.0,0.0400,...,"32.4609764, -84.9877094",columbus,GA,32.460976,-84.987709,3,2,2.0,0.0,columbus


In [ ]:
list = ['columbus','seattle']

for x in list:
    df_all['test'] = 

In [7]:
states_abbrev = {
        'AK': 'Alaska',
        'AL': 'Alabama',
        'AR': 'Arkansas',
        'AS': 'American Samoa',
        'AZ': 'Arizona',
        'CA': 'California',
        'CO': 'Colorado',
        'CT': 'Connecticut',
        'DC': 'District of Columbia',
        'DE': 'Delaware',
        'FL': 'Florida',
        'GA': 'Georgia',
        'GU': 'Guam',
        'HI': 'Hawaii',
        'IA': 'Iowa',
        'ID': 'Idaho',
        'IL': 'Illinois',
        'IN': 'Indiana',
        'KS': 'Kansas',
        'KY': 'Kentucky',
        'LA': 'Louisiana',
        'MA': 'Massachusetts',
        'MD': 'Maryland',
        'ME': 'Maine',
        'MI': 'Michigan',
        'MN': 'Minnesota',
        'MO': 'Missouri',
        'MP': 'Northern Mariana Islands',
        'MS': 'Mississippi',
        'MT': 'Montana',
        'NA': 'National',
        'NC': 'North Carolina',
        'ND': 'North Dakota',
        'NE': 'Nebraska',
        'NH': 'New Hampshire',
        'NJ': 'New Jersey',
        'NM': 'New Mexico',
        'NV': 'Nevada',
        'NY': 'New York',
        'OH': 'Ohio',
        'OK': 'Oklahoma',
        'OR': 'Oregon',
        'PA': 'Pennsylvania',
        'PR': 'Puerto Rico',
        'RI': 'Rhode Island',
        'SC': 'South Carolina',
        'SD': 'South Dakota',
        'TN': 'Tennessee',
        'TX': 'Texas',
        'UT': 'Utah',
        'VA': 'Virginia',
        'VI': 'Virgin Islands',
        'VT': 'Vermont',
        'WA': 'Washington',
        'WI': 'Wisconsin',
        'WV': 'West Virginia',
        'WY': 'Wyoming'
}

In [8]:
states_abbrev_2 = {v.lower(): k for k, v in states_abbrev.items()}

states_abbrev_2

{'alaska': 'AK',
 'alabama': 'AL',
 'arkansas': 'AR',
 'american samoa': 'AS',
 'arizona': 'AZ',
 'california': 'CA',
 'colorado': 'CO',
 'connecticut': 'CT',
 'district of columbia': 'DC',
 'delaware': 'DE',
 'florida': 'FL',
 'georgia': 'GA',
 'guam': 'GU',
 'hawaii': 'HI',
 'iowa': 'IA',
 'idaho': 'ID',
 'illinois': 'IL',
 'indiana': 'IN',
 'kansas': 'KS',
 'kentucky': 'KY',
 'louisiana': 'LA',
 'massachusetts': 'MA',
 'maryland': 'MD',
 'maine': 'ME',
 'michigan': 'MI',
 'minnesota': 'MN',
 'missouri': 'MO',
 'northern mariana islands': 'MP',
 'mississippi': 'MS',
 'montana': 'MT',
 'national': 'NA',
 'north carolina': 'NC',
 'north dakota': 'ND',
 'nebraska': 'NE',
 'new hampshire': 'NH',
 'new jersey': 'NJ',
 'new mexico': 'NM',
 'nevada': 'NV',
 'new york': 'NY',
 'ohio': 'OH',
 'oklahoma': 'OK',
 'oregon': 'OR',
 'pennsylvania': 'PA',
 'puerto rico': 'PR',
 'rhode island': 'RI',
 'south carolina': 'SC',
 'south dakota': 'SD',
 'tennessee': 'TN',
 'texas': 'TX',
 'utah': 'UT',
 